# Extract local variables



In [1]:
import locals_variables

locals_variables.main(first_year_train=1950,
         last_year_train=2010,
         first_year_test=2011,
         last_year_test=2022,
         local_latitude=45.93,
         local_longitude=9.22,
         first_clima=1980,
         last_clima=2010,
         path_predictors='C:/Users/CPR_/OneDrive - Universidad de Alcala/Documentos/GitHub/ML2EE/Processed_data/Resolution_0_5/',
         path_output='./Data/Paper/')


C:\Users\CPR_\AppData\Roaming\Python\Python39\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
